# Data Collection from json file
* Afer the search data from youtube "WebScrapin", will safe json file
* Get this file and read and understand about data

In [1]:
import pandas as pd
import numpy as np
import re
import time

import requests as rq
import bs4 as bs4
import json
import tqdm
import glob

In [2]:
df = pd.read_json('./parsed_videos.json', lines=True)

In [3]:
df.head()

,link,title,query
0,/watch?v=zyTDEa-ReVg,What you always wanted to know about Deep Lear...,machine+learning
1,/watch?v=L9daqcnxu-U,Machine Learning Experiments Management with N...,machine+learning
2,/watch?v=dnjTxUkp0Ag,Bridging Game Theory and Deep Learning Opening...,machine+learning
3,/watch?v=XwkW6dtXJCQ,Trade-offs in Machine Learning - Krishna Gogineni,machine+learning
4,/watch?v=apcQ49BpDD8,How To Debug Deep Learning Programs | A Simple...,machine+learning


In [4]:
df.describe()

,link,title,query
count,1864,1864,1864
unique,1734,1729,3
top,/watch?v=C-4EZOxM0SU,"Data Science Now - #5 ""Math for Machine Learning""",data+science
freq,4,4,650


In [5]:
links_list = df['link'].unique()

In [ ]:
url = 'https://youtube.com{link}'

for link in links_list:
    urll = url.format(link=link)
    print(urll)
    response = rq.get(urll)
    
    link_name = re.search('v=(.*)', link).group(1)
    print(link_name)
    
    with open('./dados_brutos/video_{}.html'.format(link_name), 'w+', encoding='utf-8') as output:
        output.write(response.text)
    time.sleep(2)

# Data Processing

In [3]:
with open("parsed_video_info.json", 'w+', encoding='utf-8') as output:
    for video_file in tqdm.tqdm_notebook(sorted(glob.glob("./dados_brutos/video*"))):
        with open(video_file, 'r+', encoding='utf-8') as inp:
            page_html = inp.read()
            parsed = bs4.BeautifulSoup(page_html, 'html.parser')

            class_watch = parsed.find_all(attrs={"class":re.compile(r"watch")})
            id_watch = parsed.find_all(attrs={"id":re.compile(r"watch")})
            channel = parsed.find_all("a", attrs={"href":re.compile(r"channel")})
            meta = parsed.find_all("meta")


            data = dict()

            for e in class_watch:
                colname = "_".join(e['class'])
                if "clearfix" in colname:
                    continue
                data[colname] = e.text.strip()

            for e in id_watch:
                colname = e['id']
                #if colname in output:
                #    print(colname)
                data[colname] = e.text.strip()

            for e in meta:
                colname = e.get('property')
                if colname is not None:
                    data[colname] = e['content']

            for link_num, e in enumerate(channel):
                data["channel_link_{}".format(link_num)] = e['href']


            output.write("{}\n".format(json.dumps(data)))

In [4]:
len(data)

119

In [31]:
df.shape

(1864, 3)

In [23]:
df = pd.read_json('parsed_video_info.json', lines=True)
df.shape

(1734, 173)

In [24]:
# in this case will be to see each column and analise which columns to chose what is the choices and create a filter
# set_option is a function that limeted a horizontal view.
pd.set_option('display.max_columns', 173)


In [26]:
# After know each columns, toke note and created a list of a bagging important features
data_select = ['watch-title', 'watch-view-count', 'watch-time-text', 'content_watch-info-tag-list', 'watch7-headline',
                    'watch7-user-header', 'watch8-sentiment-actions', "og:image", 'og:image:width', 'og:image:height',
                    "og:description", "og:video:width", 'og:video:height', "og:video:tag", 'channel_link_0']

In [28]:
df[data_select].head(2)

,watch-title,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
0,Data Science & Psychology as an approach in Us...,NaN,Publicado em 12 de fev. de 2020,Pessoas e blogs,Data Science & Psychology as an approach in Us...,Humans of Analytics\n\n\n\n\n\n\n\n\n\n\n\n\n\...,NaN,https://i.ytimg.com/vi/--X0GVq5A3U/hqdefault.jpg,480.0,360.0,"With the steaming conversations going around, ...",1280.0,720.0,#AnalyticsCommunity #HumansofAnalytics #DataSc...,/channel/UCgOkTQqzOWNxqFpXymQE7RA
1,Data Science Interview for a Job,565 visualizações,Publicado em 5 de nov. de 2019,Pessoas e blogs,#datascience #datascientist #interview\n\n\n\n...,Przemek Chojecki\n\n\n\n\n\n\n\n\n\n\n\n\n\nCa...,565 visualizações\n\n\n\n\n\n\n\n21\n\nGostou ...,https://i.ytimg.com/vi/-26pXGRGXeI/maxresdefau...,1280.0,720.0,How to conduct a Data Science Job Interview? H...,1280.0,720.0,get a job,/channel/UCEq0oQsS-voRnSWbcviIDGA


In [72]:
df[data_select].to_feather('raw_data.feather')

In [31]:
df[data_select].head(2)

,watch-title,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
0,Data Science & Psychology as an approach in Us...,NaN,Publicado em 12 de fev. de 2020,Pessoas e blogs,Data Science & Psychology as an approach in Us...,Humans of Analytics\n\n\n\n\n\n\n\n\n\n\n\n\n\...,NaN,https://i.ytimg.com/vi/--X0GVq5A3U/hqdefault.jpg,480.0,360.0,"With the steaming conversations going around, ...",1280.0,720.0,#AnalyticsCommunity #HumansofAnalytics #DataSc...,/channel/UCgOkTQqzOWNxqFpXymQE7RA
1,Data Science Interview for a Job,565 visualizações,Publicado em 5 de nov. de 2019,Pessoas e blogs,#datascience #datascientist #interview\n\n\n\n...,Przemek Chojecki\n\n\n\n\n\n\n\n\n\n\n\n\n\nCa...,565 visualizações\n\n\n\n\n\n\n\n21\n\nGostou ...,https://i.ytimg.com/vi/-26pXGRGXeI/maxresdefau...,1280.0,720.0,How to conduct a Data Science Job Interview? H...,1280.0,720.0,get a job,/channel/UCEq0oQsS-voRnSWbcviIDGA


In [73]:
df[data_select].to_csv('raw_data_without_labels.csv')

In [74]:
df = pd.read_csv("raw_data_without_labels.csv")

In [75]:
# Create a list to be a labels, 0= deslike and 1= like
list_labels = [0,0,0,1,0,1,1,1,1,0,0,1,1,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,
              0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,
              0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,
              0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,
              1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,
              0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
              0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,]

In [80]:
#Create a new column to be a labe 'y' with created list
pd.set_option('display.max_rows', 500)
df['y'] = pd.Series(list_labels)

In [83]:
df.to_csv('raw_data_with_labels.csv')